# Steric Sea Level Contribution from ORAS5 Reanalysis to Global Sea Level Observed by Satellite Altimetry

## Import libraries

In [ ]:
import gsw_xarray as gsw
import xarray as xr
from c3s_eqc_automatic_quality_control import download

## Define parameters

In [ ]:
year_start = 2004
year_stop = 2004  # 2023

## Define Requests

In [ ]:
time_request = {
    "year": [str(year) for year in range(year_start, year_stop + 1)],
    "month": [f"{month:02d}" for month in range(1, 13)],
}

request_reanalysis = (
    "reanalysis-oras5",
    {
        "product_type": ["consolidated", "operational"],
        "vertical_resolution": "all_levels",
        "variable": ["potential_temperature", "salinity"],
    }
    | time_request,
)
request_satellite = (
    "satellite-sea-level-global",
    {"variable": ["monthly_mean"], "version": "vdt2024"} | time_request,
)

download_kwargs = {
    "chunks": {"year": 1, "month": 1},
    "drop_variables": ["time_counter_bnds"],
}

## Define functions to cache

In [ ]:
def compute_insitu_density(ds):
    p = gsw.p_from_z(-ds["deptht"], ds["latitude"])
    SA = gsw.SA_from_SP(ds["vosaline"], p, ds["longitude"], ds["latitude"])
    CT = gsw.CT_from_pt(SA, ds["votemper"])
    rho = gsw.rho(SA, CT, p)
    return xr.merge([p, SA, CT, rho])

## Download and transform reanalysis

In [ ]:
ds_reanalysis = download.download_and_transform(
    *request_reanalysis,
    transform_func=compute_insitu_density,
    **download_kwargs,
)

## Download and transform satellite

In [ ]:
ds_satellite = download.download_and_transform(*request_satellite, **download_kwargs)